# Jira How-to

This notebook walks through how to use the jira python package to work with our Jira instance.

## Obtain an API token for Jira

- Go to https://issues.redhat.com and log in.
- Click on your profile icon in the top right corner and select "Profile".
- Click on "Personal Access Tokens" in the left sidebar.
- Click on "Create token"
- Give the token a name, and an expiration date (optional, but recommended)
- Click "Create"
- Copy the token to a safe place. You will not be able to see it again.

NOTE: This token has the same permissions as your account, so keep it safe.

You will need to set it as an environment variable. The easiest way to do this
is to add the following to a file called `.env` in the current directory:

```shell
JIRA_TOKEN=xxxxxxxxxxxxxxxxxxxxxxx
```

----

## Let's play with Atlassian's Jira API...

https://atlassian-python-api.readthedocs.io/index.html

This package is a wrapper around the Jira API, and it isn't properly typed... The calls just return dictionaries.

In [ ]:
# Create a JIRA client
from os import environ
from atlassian import Jira

jira_api_token = environ.get("JIRA_TOKEN", "")
jira_url = environ.get("JIRA_URL", "")
jira = Jira(url=jira_url, token=jira_api_token, backoff_and_retry=True)

In [ ]:
# Lets look at all the possible Jira fields and their types:
fields = jira.get_all_fields()
for field in sorted(fields, key=lambda x: x["id"]):
    ftype = field["schema"]["type"] if "schema" in field else "unknown"
    print(f"{field['id']} -> {field['name']} -- {ftype}")

In [ ]:
from pprint import pprint

for field in fields:
    if field["id"] in [
        "customfield_12311140",
        "customfield_12311141",
        "customfield_12313140",
        "customfield_12318341",
    ]:
        pprint(field)

In [ ]:
# Accessing "Parent Link" custom field
et85 = jira.get_issue("OCTOET-85")
pprint(et85["fields"]["customfield_12313140"])  # Has parent

stor919 = jira.get_issue("RHSTOR-919")
pprint(stor919["fields"]["customfield_12313140"])  # No parent

In [ ]:
interesting_fields = [
    "customfield_12311140",
    "customfield_12311141",
    "customfield_12313140",
]
issue = jira.get_issue("OPRUN-3254")
for field in interesting_fields:
    field_name = "unknown"
    for idx in fields:
        if idx["id"] == field:
            field_name = idx["name"]
            break
    print(f"{field_name} -> {issue['fields'].get(field, None)}")

In [ ]:
# Get a list of the custom fields in this issue by finding all fields that start with "customfield_"
custom_fields = [k for k in issue["fields"].keys()
                 if k.startswith("customfield_")]
for field in custom_fields:
    if issue["fields"].get(field, None) is None:
        continue
    field_name = "unknown"
    id = "?"
    for idx in fields:
        if idx["id"] == field:
            id = idx["id"]
            field_name = idx["name"]
            break
    print(f"{field_name} ({id}) -> {issue['fields'].get(field, None)}")

In [ ]:
pprint(
    jira.jql(
        "labels = 'AISummary' ORDER BY created DESC",
        limit=5,
        fields="key,summary,updated",
    )
)

In [ ]:
# With a private comment
pprint(jira.issue("OHSS-34055"))

Private comments have: `comment['visibility'] = {'type': 'group', 'value': 'Red Hat Employee'}`

Otherwise, they do not have the visibility key.

In [ ]:
pprint(jira.get_issue("OCTO-2")["fields"]["customfield_12315950"])

In [ ]:
import jiraissues

# re-import jiraissues
import importlib

jiraissues = importlib.reload(jiraissues)

i = jiraissues.Issue(jira, "OCTO-2")
for user in i.contributors:
    print(user)


print(f"\n\nI am: {jiraissues.get_self(jira)}")

In [ ]:
import summarizer
summarizer = importlib.reload(summarizer)

i = jiraissues.issue_cache.get_issue(jira, "OCTOET-77")
print(summarizer.is_active(i, 14))
print(summarizer.is_active(i, 14, True))

In [ ]:
from jiraissues import descendants, issue_cache
import rollup_status
rollup_status = importlib.reload(rollup_status)

i = jiraissues.issue_cache.get_issue(jira, "OCTOET-85")
dkeys = descendants(jira, i.key)
print(dkeys)

cats = rollup_status.categorize_issues(
    {issue_cache.get_issue(jira, k) for k in dkeys}, 14)
pprint(cats)

In [ ]:
# Experimenting w/ paged results

page_size = 100
keys = []
while True:
    print(f"Starting at {len(keys)}")
    res = jira.jql(
        "labels = 'AISummary' ORDER BY created DESC",
        limit=page_size,
        start=len(keys),
        fields="key,summary,updated",
    )
    keys.extend([issue["key"] for issue in res["issues"]])
    if len(keys) >= res["total"]:
        break

print(keys)
print(res["total"], len(keys))

In [ ]:
pprint(jira.get_issue("OCTOET-85")["fields"]["issuelinks"][1])

Based on the above, related issues provide the following information:

- type: The type of the link
    - inward: "is related to"
    - name: "Related"
    - outward: "relates to"
- inwardIssue/outwardIssue: The info about the other issue. Only one of these keys will be defined
    - fields.issuetype.name: The type of the issue (e.g., "Task")
    - fields.priority.name: The priority of the issue (e.g., "Undefined")
    - fields.status
        - name: The status of the issue (e.g., "Closed")
        - statusCategory.name: "Done"
    - fields.summary: The title of the issue
    - key: The issue key (e.g., "ABC-1234")

In [ ]:
res = jira.get_issue("OCTOET-270")["fields"]["subtasks"][0]
pprint(res)
print(res["fields"]["issuetype"]["name"])
print(res["fields"]["priority"]["name"])
print(res["fields"]["status"]["name"])
print(res["fields"]["status"]["statusCategory"]["name"])
print(res["fields"]["summary"])

In [ ]:
res = jira.get_issue("OCTOET-270")["fields"]['customfield_12311140']
pprint(res)

for r in jira.get_issue("OCTOET-270")["fields"]["issuelinks"]:
    pprint(r['type'])
print()
for r in jira.get_issue("OCTOET-85")["fields"]["issuelinks"]:
    pprint(r['type'])

In [ ]:
res = jira.get_issue("XKVMNINE-2984")["fields"]
pprint(res)

print("Feature link:")
pprint(res["customfield_12318341"])

print("Parent link:")
pprint(res["customfield_12313140"])

print("Epic link:")
pprint(res["customfield_12311140"])

In [ ]:
# page_size = 100
# keys = []
# while True:
#     print(f"Starting at {len(keys)}")
#     res = jira.jql(
#         "updated >= '2024-07-12 16:40' ORDER BY updated ASC",
#         limit=page_size,
#         start=len(keys),
#         fields="key,summary,updated",
#     )
#     keys.extend([issue["key"] for issue in res["issues"]])
#     if len(keys) >= res["total"]:
#         break

# print(keys)
# print(res["total"], len(keys))

In [ ]:
import jiraissues
import importlib
jiraissues = importlib.reload(jiraissues)

i: jiraissues.Issue = jiraissues.Issue(jira, "XCMSTRAT-31")
for r in i.related:
    print(r)

print(i.parent)

pprint(i.__dict__)

In [ ]:
from jiraissues import CF_EPIC_LINK, CF_PARENT_LINK, rget


issue = jira.get_issue("OCTOET-353")
pprint(issue)
print(rget(issue, "fields", CF_EPIC_LINK, default="zzz"))
print(rget(issue, "fields", CF_PARENT_LINK))
print(rget(issue, "fields", "parent", "key"))